In [1]:
# assert 3 >= 1 + 3
import torch

In [3]:
ts = torch.tensor([1, 2, 3]).unsqueeze(0).cuda()

In [4]:
ts

tensor([[1, 2, 3]], device='cuda:0')

In [23]:
from functools import partial
import os
import sys
# ROOT_DIR = '../'
# sys.path.append(os.path.join(ROOT_DIR, 'models'))
from models.helpers import GenericMLP

import torch

In [3]:
decoder_dim=256
mlp_dropout = 0.3

mlp_func = partial(
    GenericMLP,
    norm_fn_name="bn1d",
    activation="relu",
    use_conv=True,
    hidden_dims=[decoder_dim, decoder_dim],
    dropout=mlp_dropout,
    input_dim=decoder_dim,
)
dataset_config_num_semcls = 5
semcls_head = mlp_func(output_dim=dataset_config_num_semcls + 1)

In [4]:
semcls_head

GenericMLP(
  (layers): Sequential(
    (0): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Conv1d(256, 6, kernel_size=(1,), stride=(1,))
  )
)

In [32]:
semcls_head.layers[-1].in_channels

256

In [13]:
semcls_head.layers[-1].bias

Parameter containing:
tensor([ 5.4622e-02,  2.7667e-02,  5.5343e-02, -3.0710e-02,  4.5826e-02,
         7.5258e-05], requires_grad=True)

In [11]:
_b = semcls_head.layers[-1].bias.detach().clone()

In [17]:
print(_b.shape)
print(_b[:-1].shape)

torch.Size([6])
torch.Size([5])


In [20]:
_w = semcls_head.layers[-1].weight.detach().clone()
print(_w.shape)
print(_w[:-1, ...].shape)

torch.Size([6, 256, 1])
torch.Size([5, 256, 1])


In [21]:
_b_no_bg = _b[:-1]
_w_no_bg = _w[:-1, ...]

In [26]:
_w_new = torch.cat((_w_no_bg, _w), dim=0)
print(_w_new.shape)

_b_new = torch.cat((_b_no_bg, _b), dim=0)
print(_b_new.shape)

torch.Size([11, 256, 1])
torch.Size([11])


In [ ]:
# Conclusion: change the last conv1d of the GenericMLP to a new size.
# Note that the last class is for "no object" class.